In [1]:
from pathlib import Path
import sqlparse

def construct_query(site, sql_file, placeholders):
    # Read the query file
    project_root = get_project_root_folder()
    query_file_path = Path(f"{project_root}/{site}/sql/{sql_file}").absolute()
    with open(query_file_path.absolute(), "r") as f:
        query = f.read().replace("\n", " ")

    # Replace all placeholders
    for key, value in placeholders.items():
        query = query.replace(f"${key}", value.replace("'", "''"))

    # Format the query
    query = sqlparse.format(query, reindent=True, keyword_case='upper')

    return query

In [ ]:
import pandas as pd
import sqlite3
import os

DB_PATH_VARIABLES = {
    "aircraft": "FLIGHT_RECORDER_DB",
    "wildlife": "NATURE_RECORDER_DB"
}

def query_data(site, query):
    # Connect to the database, execute the query and read the results into a dataframe
    database_path = os.environ[DB_PATH_VARIABLES[site]]
    connection = sqlite3.connect(database_path)
    df = pd.read_sql_query(query, connection, parse_dates=["Date"])

    # Check there is some data
    if not df.shape[0]:
        message = f"No data found"
        raise ValueError(message)

    return df